# 참조자(References)와 빌림(Borrowing)



- 변수 s가 유효한 스코프는 여느 함수의 파라미터의 스코프와 동일하지만, 
- 소유권을 갖고 있지 않으므로 이 참조자가 스코프 밖으로 벗어났을 때 참조자가 가리키고 있는 값은 버리지 않습니다. 
- 또한 실제 값 대신 참조자를 파라미터로 갖고 있는 함수는 소유권을 갖고 있지 않기 때문에 
- 소유권을 되돌려주기 위해 값을 다시 반환할 필요도 없다는 뜻이 됩니다.

### 자료형 알아보기 

In [10]:
fn print_type_name_of<T>(_: T) {
    println!("{}", std::any::type_name::<T>() );
}

## 1. 변수의 참조자 정의 

## 1-1 변수에 직접 참조를 지정하기 

### 변수에 참조자 적용

- 직접 값에 & 지정 가능
- 변수 앞에 ref 예약어 지정 가능

In [6]:
fn main() {
    let var = &100;
    let ref r_var = 100; 
    
    assert_eq!(var,r_var);
    
    println!(" 잘 처리됨 ");
}

In [7]:
main()

 잘 처리됨 


()

### 변수에 ref 예약어 지정해서 참조자를 사용히기 

In [13]:
fn main() {
    let x1 = &false;            // 참조값을 받는 x1
    print_type_name_of(x1);
    println!(" {} ", x1);

    let &x2 = &false;           // 참조값을 받는 변수 x2를 지정 
    print_type_name_of(&x2);
     println!(" {} ", &x2);
    
    assert_eq!(x1,&x2);         // 동일한 타입이다. 

    let ref x3 = &false;        // 두번의 참조값을 받는 변수 x3가 생김
    print_type_name_of(x3);
    println!(" {} ", x3);
}



In [14]:
main();

&bool
 false 
&bool
 false 
&&bool
 false 


## 1-2  변경 가능한 참조를 변수에 지정하기 

### 변경가능

- 변경가능한 변수 이름앞에 mut를 지정해서 변수를 변경시킬 수 있다.
- 값을 변경가능한 참조로 지정해서 해당 변수의 값을 변경할 수 있다. 
- 이때 ref mut를 동시에 사용해서 저당되는 변수의 참조를 얻을 수도 있다

In [15]:
fn main() { 
    let mut x = 100; 
    let y = &mut x;
    *y = 300;
    println!(" {}", x);
    
    let ref mut y1 = x;
    *y1 = 999;
    println!(" {}", x);
}

In [16]:
main();

 300
 999


## 1-3 mut 를 어디에 지정하느냐에 따라 의미가 조금 차이가 있다. 

- let mut ref_x: &i32와 let ref_x: &mut i32의 차이점에 주의 하시기 바랍니다. 
- 첫번째 값은 다른 값에 바인딩 될 수 있는 가변 참조이고, 두번째 값은 가변 값에 대한 참조입니다.

### 가변 값에 대한 참조 

In [2]:
fn main() {
    let mut x: i32 = 10;
    let ref_x: &mut i32 = &mut x;
    *ref_x = 20;
    println!("x: {x}");
}

In [3]:
main();

x: 20


### 참조를 저장하는 가변변수 처리 

In [8]:
fn main() {
    let mut x: i32 = 10;
    let mut ref_x: &i32 = &x;
    //*ref_x = 20;           //cannot assign to `*ref_x`, which is behind a `&` reference
    ref_x = &20;
    println!("x: {x}");
}

In [9]:
main();

x: 10


## 2 함수를 사용한 참조 처리 

## 2-1. 함수의 매개변수에 참조 문자열을 지정

- 이 함수는 참조문자열의 길이를 반환한다.

In [5]:
fn calculate_length(s: &String) -> usize { // s는 String의 참조자입니다
    s.len()
}                                          // 여기서 s는 스코프 밖으로 벗어났습니다. 하지만 가리키고 있는 값에 대한 소유권이 없기
                                           // 때문에, 아무런 일도 발생하지 않습니다.

### 문자열을 정의하고 문자열의 길이를 

In [8]:
fn main() {
    let s1 = String::from("hello");

    let len = calculate_length(&s1);        // 문자열을 참조로 전달한다. 

    println!("문자열은 {} 길이는 {}.", s1, len);
}

In [9]:
main()

문자열은 hello 길이는 5.


()

## 2-2. 참조는 갱신이 불가하다

- 함수에 참조 매개변수를 지정한 경우 이 매개변수를 갱신할 수 없다.

In [12]:
fn change(some_string: &String) {
    some_string.push_str(", world");
}

fn main() {
    let s = String::from("hello");

    change(&s);
}

Error: cannot borrow `*some_string` as mutable, as it is behind a `&` reference

## 2-3 갱신은 가변 참조를 사용한다

- 가변 참조자는 딱 한가지 큰 제한이 있습니다: 특정한 스코프 내에 특정한 데이터 조각에 대한 가변 참조자를 딱 하나만 만들 수 있다는 겁니다

In [12]:
fn main() {
    let mut s = String::from("hello");   // 갱신가능한 변수 지정 

    change(&mut s);
    
    println!("s = {}", s)
}

fn change(some_string: &mut String) {    // 갱신가능한 매개변수 지정 
    some_string.push_str(", world");
}

In [13]:
main()

s = hello, world


()

##  3. 스코프 별로 가변 참조는 하나만 사용이 가능하다

- 가변 참조는 같은 영역에 하나만 있어야 함  하나만 가변으로 만들 수 있다. 

### 두 개의 가변참조를 변수에 저장하고 갱신할 수 없다.

- 같은 영역에서는 항상 동일한 소유권에는 하나의 가변참조만 가능

In [15]:
let mut s = String::from("hello");

{
    let r1 = &mut s;
    let r2 = &mut s;
    
    r1.push_str(", world");
    r2.push_str(", world !!!!");
}

Error: cannot borrow `s` as mutable more than once at a time

### 위의 문제를 해결하려면 

In [17]:
let mut s = String::from("hello");

{
    let _r1 = &mut s;
    _r1.push_str(", world");

} // 여기서 r1은 스코프 밖으로 벗어났으므로, 우리는 아무 문제 없이 새로운 참조자를 만들 수 있습니다.

{
    let _r2 = &mut s;
    _r2.push_str("!!!!");
}

()

In [18]:
println!(" {}", s);

 hello, world!!!!


In [ ]:
### 

In [37]:

fn main() { 
    let mut s1 = String::from("hello");

    let r11 = &s1; // 문제 없음
    let _r22 = &r11; // 문제 없음
    let r33 = &mut s1; 
    r33.push_str(", world");
    println!(" {} ", s1);

}

In [38]:
main()

 hello, world 


()

## 6. 댕글링 참조 

In [32]:
fn main() {
    let reference_to_nothing = dangle();
}

fn dangle() -> &String {           // dangle은 String의 참조자를 반환합니다

    let s = String::from("hello"); // s는 새로운 String입니다

    &s                             // 우리는 String s의 참조자를 반환합니다.
}                                  // 여기서 s는 스코프를 벗어나고 버려집니다. 이것의 메모리는 사라집니다.

Error: missing lifetime specifier

## 위의 해법 

In [33]:
fn main() {
    let reference_to_nothing = dangle();
}

fn dangle() -> String {             // dangle은 String의 참조자를 반환합니다

    let s = String::from("hello");  // s는 새로운 String입니다

    s                               // 우리는 String s의 참조자를 반환합니다.
}                                   // 여기서 s는 스코프를 벗어나고 버려집니다. 이것의 메모리는 사라집니다.

In [34]:
main()

()

## 러스트는 죽은 참조를 허용하지 않는다. 

- 참조는 어떤 값을 “빌리는” 것입니다.
- 러스트는 참조 대상의 값이, 그 값에 대한 모든 참조들보다 더 오래 살아있음을 추적합니다.


In [11]:
fn main() {
    let ref_x: &i32;
    {
        let x: i32 = 10;
        ref_x = &x;                 // 소유자 x 보다 참조를 더 오래동안 사용할 수 없다 
    }                               // 그래서 댕글링 참조를 컴파이러에서 체크 
    println!("ref_x: {ref_x}");
}

Error: `x` does not live long enough